In [1]:
## reload submodule
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

import qkit as qkit

In [3]:
data = pd.read_csv("./data/_^spx.csv", index_col=0, parse_dates=[0], sep=",")
data

,open,high,low,close,volume,signal
date,,,,,,
2023-01-03,3853.290039,3878.459961,3794.330078,3824.139893,3959140000,1
2023-01-04,3840.360107,3873.159912,3815.770020,3852.969971,4414080000,1
2023-01-05,3839.739990,3839.739990,3802.419922,3808.100098,3893450000,1
2023-01-06,3823.370117,3906.189941,3809.560059,3895.080078,3923560000,1
2023-01-09,3910.820068,3950.570068,3890.419922,3892.090088,4311770000,1
...,...,...,...,...,...,...
2025-04-01,5597.529785,5650.569824,5558.520020,5633.069824,4434500000,-1
2025-04-02,5580.759766,5695.310059,5571.479980,5670.970215,4243830000,-1
2025-04-03,5492.740234,5499.529785,5390.830078,5396.520020,7210470000,-1


In [4]:
backtest_params = {
    "initial_balance": 10000,
    "default_entry_value": 0.9,
    "default_entry_value_max": 20000,
}

bt = qkit.Backtester(dataset=data, **backtest_params)

for broker in bt.run():
    current_bar = broker.state.data.iloc[-1]
    previous_bar = broker.state.data.iloc[-2]

    current_signal = current_bar["signal"]
    previous_signal = previous_bar["signal"]
    changed = current_signal != previous_signal

    if changed:
        if current_signal == 1:
            broker.buy()
        elif current_signal == -1:
            broker.sell()
        else:
            broker.close()

In [5]:
trades = bt.trades()
trades.columns

Index(['is_long', 'entry_value', 'entry_price', 'entry_bar',
       'entry_commission', 'entry_time', 'exit_value', 'exit_price',
       'exit_commission', 'exit_bar', 'exit_time', 'pnl', 'return_pct'],
      dtype='object')

In [6]:
trades[["is_long", "entry_time", "entry_price", "exit_price", "pnl", "return_pct"]]

,is_long,entry_time,entry_price,exit_price,pnl,return_pct
0,False,2023-02-02,4179.759766,3861.590088,734.125519,0.082393
1,True,2023-03-10,3861.590088,4588.959961,1785.310519,0.188360
2,False,2023-07-31,4588.959961,4117.370117,1256.298938,0.114537
3,True,2023-10-27,4117.370117,5254.350098,3307.899923,0.276142
4,False,2024-03-28,5254.350098,4967.229980,854.910194,0.057803
5,True,2024-04-19,4967.229980,5667.200195,2171.882762,0.140918
6,False,2024-07-16,5667.200195,5186.330078,1593.919059,0.092719
7,True,2024-08-05,5186.330078,5648.399902,1643.220004,0.089094
8,False,2024-08-30,5648.399902,5408.419922,875.399353,0.044372
9,True,2024-09-06,5408.419922,6090.270020,2496.224799,0.126072
